In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [0]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
var defaultVarsBE = await Workspace.Query<IfrsVariable>().ToArrayAsync();
await Workspace.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync()

In [0]:
var yieldCurve_Test = @"
@@YieldCurve
Currency,Year,Month,Scenario,Values0,Values1,Values2,Values3
USD,2021,6,,0.003,0.003,0.003,0.003
USD,2021,6,MTUP10pct,0.103,0.103,0.103,0.103";

In [0]:
var cashflows_BestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2021,6,
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,CL,C,,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,CU,CL,C,,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-3.3
DT1.1,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";

In [0]:
var actuals_BestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2021,6,
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,-3008";

In [0]:
var argsBestEstimate = new ImportArgs ("CH", 2021, 6, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsScenario = new ImportArgs ("CH", 2021, 6, Periodicity.Quarterly, "MTUP10pct", ImportFormats.Cashflow);

In [0]:
var yieldCurveLog = await Import.FromString(yieldCurve_Test).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));

# Import Cashflow

In [0]:
await Import.FromString(cashflows_BestEstimate).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNode>(argsBestEstimate);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var rawVarsBE = await DataSource.Query<RawVariable>().ToArrayAsync();
var ifrsVarsBE = await DataSource.Query<IfrsVariable>().ToArrayAsync();
rawVarsBE.Length.Should().NotBe(0);
ifrsVarsBE.Length.Should().NotBe(0);

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNode>(argsScenario);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var rawVarsSCE = await DataSource.Query<RawVariable>().ToArrayAsync();
var ifrsVarsSCE = await DataSource.Query<IfrsVariable>().ToArrayAsync();
rawVarsSCE.Length.Should().Be(0);
ifrsVarsSCE.Length.Should().NotBe(0);

In [0]:
ifrsVarsSCE.Where(x => x.EconomicBasis == "C").Intersect(ifrsVarsBE.Where(x => x.EconomicBasis == "C"), IfrsVariableComparer.Instance()).ToArray().Length.Should().Be(0);

In [0]:
ifrsVarsSCE

In [0]:
ifrsVarsBE

# Import Actuals

In [0]:
await Import.FromString(cashflows_BestEstimate).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNode>(argsBestEstimate);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var newRawVarsBE = await DataSource.Query<RawVariable>().ToArrayAsync();
var newIfrsVarsBE = await DataSource.Query<IfrsVariable>().ToArrayAsync();
newRawVarsBE.Length.Should().NotBe(0);
newIfrsVarsBE.Length.Should().NotBe(0);

In [0]:
newIfrsVarsBE

In [0]:
newIfrsVarsBE.Except(ifrsVarsBE, IfrsVariableComparer.Instance()).ToArray().Length.Should().NotBe(0);

In [0]:
newIfrsVarsBE

In [0]:
ifrsVarsBE

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNode>(argsScenario);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var newRawVarsSCE = await DataSource.Query<RawVariable>().ToArrayAsync();
var newIfrsVarsSCE = await DataSource.Query<IfrsVariable>().Where(x => x.EconomicBasis == "C").ToArrayAsync();
newRawVarsSCE.Length.Should().Be(0);
newIfrsVarsSCE.Length.Should().NotBe(0);

In [0]:
ifrsVarsSCE.Intersect(newIfrsVarsSCE, IfrsVariableComparer.Instance()).ToArray().Length.Should().Be(0);

# Debug

In [0]:
(Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(mainArgs))

In [0]:
var extension = System.IO.Path.GetExtension(file);
var stream = await Project.FileStorage.ReadAsync(file);
var dataSet = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet;

    Activity.Start();
    var mainArgs = (await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(dataSet)) with {ImportFormat = ImportFormats.Cashflow};
    if(Activity.HasErrors()) return Activity.Finish();
    await DataNodeFactoryAsync(dataSet, ImportFormats.Cashflow, mainArgs);
    if(Activity.HasErrors()) return Activity.Finish();
    
    var workspace = Workspace.CreateNew();
    var log = await ParseCashflowsToWorkspaceAsync(dataSet, mainArgs, workspace);
    if(log.Errors.Any()) return Activity.Finish().Merge(log);

    var targetArgs = mainArgs.Scenario == null  
                     ? mainArgs.RepeatOnce().Concat(await DataSource.Query<PartitionByReportingNodeAndPeriod>()  // TODO: check that concat is adding at the end of the list
                            .Where(x => x.ReportingNode == mainArgs.ReportingNode && x.Year == mainArgs.Year && x.Month == mainArgs.Month && x.Scenario != null)
                            .Select(x => new ImportArgs(x.ReportingNode, x.Year, x.Month, default(Periodicity), x.Scenario, ImportFormats.Cashflow)).ToArrayAsync())
                      : mainArgs.RepeatOnce();
        
    foreach (var args in targetArgs) {
        log = log.Merge(await CalculateAndUploadIfrsVariablesAsync(args, workspace));
        if(log.Errors.Any()) return  Activity.Finish().Merge(log);
    }

    await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(mainArgs);
    await CommitToDatabase<RawVariable>(workspace, (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(mainArgs)), snapshot : true);
    return Activity.Finish().Merge(log); 
    

In [0]:
 Activity.Start();
    var args = targetArgs.ToArray()[0];
    var storage = new ImportStorage(args, DataSource, workspace);
    await storage.InitializeAsync();
    if(Activity.HasErrors()) return Activity.Finish();
   
    var universe = Scopes.ForStorage(storage).ToScope<IModel>();
    var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
    var ivs = universe.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.CalculatedIfrsVariables);
    if(Activity.HasErrors()) return Activity.Finish();
    
    await workspace.UpdateAsync<IfrsVariable>(ivs);
    await CommitToDatabase<IfrsVariable>(workspace,
                                         storage.TargetPartition, 
                                         snapshot : true, 
                                         filter : x => storage.EstimateTypesByImportFormat[ImportFormats.Cashflow].Contains(x.EstimateType) && 
                                                       storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));
    return Activity.Finish();

In [0]:
    var args = targetArgs.ToArray()[1];
    var storage = new ImportStorage(args, DataSource, workspace);
    await storage.InitializeAsync();

In [0]:
storage.TargetPartition

In [0]:
await storage.InitializeAsync()

In [0]:
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(storage.TargetPartition)

In [0]:
(workspace.Partition.GetCurrentPartitions(), DataSource.Partition.GetCurrentPartitions())

In [0]:
(workspace.Query<IfrsVariable>().Count(), DataSource.Query<IfrsVariable>().Count())

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<IfrsVariable>().ToArrayAsync() );
await DataSource.CommitAsync()

In [0]:
ivs

In [0]:
identities.First()

In [0]:
ivs

In [0]:
var extension = System.IO.Path.GetExtension(file);
var stream = await Project.FileStorage.ReadAsync(file);
var dataSet = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet;

// begin cashflow importer
    Activity.Start();
    var args = (await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(dataSet)) with {ImportFormat = ImportFormats.Cashflow};
    await DataNodeFactoryAsync(dataSet, ImportFormats.Cashflow, args);
    if(Activity.HasErrors()) return Activity.Finish();
    
    var workspace = Workspace.CreateNew();
    var parsingLog = await ParseCashflowsToWorkspaceAsync(dataSet, args, workspace);
    if(parsingLog.Errors.Any()) return Activity.Finish().Merge(parsingLog);

    /*var calculationLog = await CalculateAndSaveVarsAsync(args, workspace);
    if(calculationLog.Errors.Any()) return  Activity.Finish().Merge(parsingLog).Merge(calculationLog); already imported */
    
    //if (args.Scenario == null){
        var saveRawVars = false;
        var calculatingPartitions = await DataSource.Query<PartitionByReportingNodeAndPeriod>()
                                            .Where(x => x.ReportingNode == args.ReportingNode
                                                   && x.Year == args.Year
                                                   && x.Month == args.Month
                                                   && x.Scenario != null).ToArrayAsync();
    
       // foreach (var partition in calculatingPartitions){
            var partition = calculatingPartitions[0];
            args = new ImportArgs(partition.ReportingNode, partition.Year, partition.Month, Periodicity.Quarterly, partition.Scenario, ImportFormats.Cashflow);

            await DataNodeFactoryAsync(dataSet, ImportFormats.Cashflow, args);
            if(Activity.HasErrors()) return Activity.Finish();

            //workspace = Workspace.CreateNew();
            workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
            workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());
            // debug CalculateAndSaveVarsAsync
            //var newCalculationLog = await CalculateAndSaveVarsAsync(args, workspace, saveRawVars);
            var storage = new ImportStorage(args, DataSource, workspace);
            //await storage.InitializeAsync();
            //if(Activity.HasErrors()) return Activity.Finish();
            //debug importstorage
                int periodicityInMonths = 3;
                var hierarchyCache  = workspace.ToHierarchicalDimensionCache();
                string ImportFormat = args.ImportFormat;
                (int Year, int Month) CurrentReportingPeriod = (args.Year, args.Month);
                (int Year, int Month) PreviousReportingPeriod = (args.Year - 1, MonthInAYear); // YTD Logic
                var estimateTypes = (await workspace.Query<EstimateType>().ToArrayAsync());
        
                var EstimateTypeDimension     = estimateTypes.ToDictionary(x => x.SystemName);
                var AmountTypeDimension       = (await workspace.Query<AmountType>().ToArrayAsync()).ToDictionary(x => x.SystemName);
                var NoveltyDimension          = (await workspace.Query<Novelty>().ToArrayAsync()).ToDictionary(x => x.SystemName);
        
                var ExperienceAdjustEstimateTypeMapping = new Dictionary<string,string>{{EstimateTypes.A, EstimateTypes.APA}}; //TODO move this logic
        
                //Hierarchy Cache
                await hierarchyCache.InitializeAsync<AmountType>();
        
                //EstimateType to load and to update
                var EstimateTypesByImportFormat = new InputSource[] { InputSource.Opening, InputSource.Actual, InputSource.Cashflow,}
                                        .ToDictionary(x => x.ToString(), 
                                                      x => estimateTypes
                                                      .Where(et => et.InputSource.Contains(x))
                                                      .Select(et => et.SystemName)
                                                      .ToHashSet());
        
                //ProjectionConfiguration : Current Period + projection for every Quarter End for current Year and next Years as in projectionConfiguration.csv
                var ProjectionConfiguration = (await workspace.Query<ProjectionConfiguration>()
                                               .Where(x => x.Shift > 0 || x.TimeStep == args.Month || (x.TimeStep > args.Month && x.TimeStep % periodicityInMonths == 0))
                                               .OrderBy(x => x.Shift).ThenBy(x => x.TimeStep).ToArrayAsync());
        
                //Get Partitions
                var PartitionByRn = (await DataSource.Query<PartitionByReportingNode>().Where(p => p.ReportingNode == args.ReportingNode).ToArrayAsync()).Single().Id;
                var TargetPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                   p.Year == CurrentReportingPeriod.Year &&
                                                                                                   p.Month == CurrentReportingPeriod.Month &&
                                                                                                   p.Scenario == args.Scenario).ToArrayAsync()).Single().Id;
                var DefaultPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                    p.Year == CurrentReportingPeriod.Year &&
                                                                                                    p.Month == CurrentReportingPeriod.Month &&
                                                                                                    p.Scenario == null).ToArrayAsync()).Single().Id;
                //Set Partitions
                await DataSource.Partition.SetAsync<PartitionByReportingNode>(PartitionByRn);
                await workspace.Partition.SetAsync<PartitionByReportingNode>(PartitionByRn);
        
                await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
                await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
        
                //Get data from Workspace (result of parsing)
                var parsedRawVariables = await workspace.LoadVariablesRelaxedAsync<RawVariable>(DataSource, TargetPartition, DefaultPartition);
                var parsedIfrsVariables = await workspace.LoadVariablesRelaxedAsync<IfrsVariable>(DataSource, TargetPartition, DefaultPartition);
        
                //DataNodes
                var DataNodeDataBySystemName = await workspace.LoadDataNodesAsync(args);
        
                //Accident Years
                var AccidentYearsByDataNode = (IDictionary<string, ICollection<int?>>)
                    (ImportFormat == ImportFormats.Cashflow ? parsedRawVariables.Select(x => new {x.DataNode, x.AccidentYear}) : parsedIfrsVariables.Select(x => new {x.DataNode, x.AccidentYear}))
                    .ToDictionaryGrouped(x => x.DataNode, x => (ICollection<int?>)x.Select(y => y.AccidentYear).ToHashSet());
        
                // Import Scopes and Data Node relationship parameters
                var InterDataNodeParametersByGoc = await workspace.LoadInterDataNodeParametersAsync(args);
        
                var primaryScopeFromParsedVariables = (ImportFormat == ImportFormats.Cashflow ? parsedRawVariables.Select(x => x.DataNode) : parsedIfrsVariables.Select(x => x.DataNode)).ToHashSet();
                var primaryScopeFromLinkedReinsurance = primaryScopeFromParsedVariables
                                            .Where(goc => !DataNodeDataBySystemName[goc].IsReinsurance && DataNodeDataBySystemName[goc].LiabilityType == LiabilityTypes.LRC)
                                            .SelectMany(goc => InterDataNodeParametersByGoc.TryGetValue(goc, out var interDataNodeParamByPeriod)
                                                                  ? interDataNodeParamByPeriod[CurrentPeriod].Select(param => param.DataNode == goc ? param.LinkedDataNode : param.DataNode).Where(goc => !primaryScopeFromParsedVariables.Contains(goc))
                                                                  : Enumerable.Empty<string>())
                                            .ToHashSet();
        
                var primaryScope = primaryScopeFromParsedVariables.Concat(primaryScopeFromLinkedReinsurance).ToHashSet();
                var secondaryScope = InterDataNodeParametersByGoc
                            .Where(kvp => primaryScope.Contains(kvp.Key))
                            .SelectMany(kvp => { var linkedGocs = kvp.Value[CurrentPeriod].Select(param => param.DataNode == kvp.Key ? param.LinkedDataNode : param.DataNode);
                                                return linkedGocs.Where(goc => !primaryScope.Contains(goc));}).ToHashSet();
                var allImportScopes = new HashSet<string>(primaryScope.Concat(secondaryScope));
        
                var DataNodesByImportScope = new Dictionary<ImportScope, HashSet<string>> { { ImportScope.Primary, primaryScope }, { ImportScope.Secondary, secondaryScope } };
        
                // Parameters
                var PartnerRating = await workspace.LoadCurrentAndPreviousParameterAsync<PartnerRating>(args, x => x.Partner);
                var CreditDefaultRates = await workspace.LoadCurrentAndPreviousParameterAsync<CreditDefaultRate>(args, x => x.CreditRiskRating);
                var SingleDataNodeParametersByGoc = await workspace.LoadSingleDataNodeParametersAsync(args);

                var LockedInYieldCurve = await workspace.LoadLockedInYieldCurveAsync(args, allImportScopes.Select(dn => DataNodeDataBySystemName[dn]));
                var CurrentYieldCurve = await workspace.LoadCurrentYieldCurveAsync(args, allImportScopes.Select(dn => DataNodeDataBySystemName[dn])); //TODO Rename this variable
        
                var AocConfigurationByAocStep = await DataSource.LoadAocStepConfigurationAsDictionaryAsync(args.Year, args.Month);         
                var aocStepByInputSource = ((InputSource[])Enum.GetValues(typeof(InputSource))).ToDictionary(x => x,
                                                                                                 x => AocConfigurationByAocStep
                                                                                                 .Where(kvp => kvp.Value.InputSource.Contains(x))
                                                                                                 .Select(kvp => kvp.Key)
                                                                                                 .ToHashSet());
        



            /*var universe = Scopes.ForStorage(storage).ToScope<IModel>();
            var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
            var ivs = universe.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.CalculatedIfrsVariables);
            if(Activity.HasErrors()) return Activity.Finish();
    
            await workspace.UpdateAsync<IfrsVariable>(ivs);
            await CommitToDatabase<IfrsVariable>(workspace,
                                         storage.TargetPartition, 
                                         snapshot : true, 
                                         filter : x => storage.EstimateTypesByImportFormat[ImportFormats.Cashflow].Contains(x.EstimateType) && 
                                                       storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));
            if(saveRawVars) await CommitToDatabase<RawVariable>(workspace, 
                                        storage.TargetPartition, 
                                        snapshot : true,
                                        filter : x => storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));*/
            //if(newCalculationLog.Errors.Any()) return  Activity.Finish().Merge(parsingLog).Merge(calculationLog);
            //calculationLog = calculationLog.Merge(newCalculationLog);
            //end debug CalculateAndSaveVarsAsync
        //}
    //}
    return Activity.Finish().Merge(parsingLog); 

In [0]:
CurrentYieldCurve["USD"]

In [0]:
LockedInYieldCurve

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);

In [0]:
await workspace.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync()